# Visual Document Search with Azure Content Understanding
## Objective
This document illustrates an example workflow for how to leverage the Azure AI Content Understanding API to enhance the quality of document search.

The sample will demonstrate the following steps:
1. Extract the layout and content of a document using Azure AI Document Intelligence.
2. For each figure in the document, extract its content with a custom analyzer using Azure AI Content Understanding, and insert it into the corresponding location in the document content.
2. Chunk and embed the document content with LangChain and Azure OpenAI, and index them with Azure Search to generate an Azure Search index.
3. Utilize an OpenAI chat model to search through content in the document with a natural language query.


## Pre-requisites
1. Follow the [README](../README.md#configure-azure-ai-service-resource) to create the required resources for this sample.
1. Install the required packages.

In [ ]:
%pip install -r ../requirements.txt

## Load environment variables

In [1]:
from dotenv import load_dotenv
import os

load_dotenv(override=True)

# Load and validate Azure AI Services configs
AZURE_AI_SERVICE_ENDPOINT = os.getenv("AZURE_AI_SERVICE_ENDPOINT")
AZURE_AI_SERVICE_API_VERSION = os.getenv("AZURE_AI_SERVICE_API_VERSION") or "2024-12-01-preview"
AZURE_DOCUMENT_INTELLIGENCE_API_VERSION = os.getenv("AZURE_DOCUMENT_INTELLIGENCE_API_VERSION") or "2024-11-30"

# Load and validate Azure OpenAI configs
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_CHAT_DEPLOYMENT_NAME = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")
AZURE_OPENAI_CHAT_API_VERSION = os.getenv("AZURE_OPENAI_CHAT_API_VERSION") or "2024-08-01-preview"
AZURE_OPENAI_EMBEDDING_DEPLOYMENT_NAME = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT_NAME")
AZURE_OPENAI_EMBEDDING_API_VERSION = os.getenv("AZURE_OPENAI_EMBEDDING_API_VERSION") or "2023-05-15"

# Load and validate Azure Search Services configs
AZURE_SEARCH_ENDPOINT = os.getenv("AZURE_SEARCH_ENDPOINT")
AZURE_SEARCH_INDEX_NAME = os.getenv("AZURE_SEARCH_INDEX_NAME") or "sample-index-visual-doc"

## File to analyze

In [24]:
from pathlib import Path

# Get the path to the file that will be analyzed
# Testing with Venture Deals - a full book
file = Path("../data/Venture-deals.pdf")

## Create custom analyzer using chart and diagram understanding template

In [25]:
import json
import sys
import uuid

# Add the parent directory to the path to use shared modules
parent_dir = Path(Path.cwd()).parent
sys.path.append(str(parent_dir))
from python.content_understanding_client import AzureContentUnderstandingClient

from azure.identity import DefaultAzureCredential, get_bearer_token_provider
credential = DefaultAzureCredential()
token_provider = get_bearer_token_provider(credential, "https://cognitiveservices.azure.com/.default")

# Get path to sample template
ANALYZER_TEMPLATE_PATH = "../analyzer_templates/image_chart_diagram_understanding.json"

# Create analyzer
ANALYZER_ID = "content-understanding-search-sample-" + str(uuid.uuid4())
content_understanding_client = AzureContentUnderstandingClient(
    endpoint=AZURE_AI_SERVICE_ENDPOINT,
    api_version=AZURE_AI_SERVICE_API_VERSION,
    token_provider=token_provider,
    x_ms_useragent="azure-ai-content-understanding-python/search_with_visusal_document", # This header is used for sample usage telemetry, please comment out this line if you want to opt out.
)

try:
    response = content_understanding_client.begin_create_analyzer(ANALYZER_ID, analyzer_template_path=ANALYZER_TEMPLATE_PATH)
    result = content_understanding_client.poll_result(response)
    print(f'Analyzer details for {result["result"]["analyzerId"]}:')
    print(json.dumps(result, indent=2))
except Exception as e:
    print(e)
    print("Error in creating analyzer. Please double-check your analysis settings.\nIf there is a conflict, you can delete the analyzer and then recreate it, or move to the next cell and use the existing analyzer.")

Analyzer details for content-understanding-search-sample-2511c091-f01c-422d-a78a-f260d03c6f46:
{
  "id": "a93385eb-d3fb-4594-8547-d5868d185f60",
  "status": "Succeeded",
  "result": {
    "analyzerId": "content-understanding-search-sample-2511c091-f01c-422d-a78a-f260d03c6f46",
    "description": "Extract detailed structured information from charts and diagrams.",
    "createdAt": "2025-10-04T12:06:27Z",
    "lastModifiedAt": "2025-10-04T12:06:27Z",
    "config": {
      "returnDetails": false,
      "disableContentFiltering": false
    },
    "fieldSchema": {
      "name": "ChartsAndDiagrams",
      "fields": {
        "Title": {
          "type": "string",
          "description": "Verbatim title of the chart."
        },
        "ChartType": {
          "type": "string",
          "description": "The type of chart.",
          "enum": [
            "area",
            "bar",
            "box",
            "bubble",
            "candlestick",
            "funnel",
            "heatmap

## Analyze document layout and compose with figure descriptions

In [9]:
from azure.ai.documentintelligence import DocumentIntelligenceClient
from azure.ai.documentintelligence.models import AnalyzeResult
from azure.ai.documentintelligence.models import AnalyzeDocumentRequest
import fitz
from PIL import Image

# Define helper functions for document-figure composition
def insert_figure_contents(md_content, figure_contents, span_offsets):
    """
    Inserts the figure content for each of the provided figures in figure_contents
    before the span offset of that figure in the given markdown content.

    Args:
    - md_content (str): The original markdown content.
    - figure_contents (list[str]): The contents of each figure to insert.
    - span_offsets (list[int]): The span offsets of each figure in order. These should be sorted and strictly increasing.

    Returns:
    - str: The modified markdown content with the the figure contents prepended to each figure's span.
    """
    # NOTE: In this notebook, we only alter the Markdown content returned by the Document Intelligence API,
    # and not the per-element spans in the API response. Thus, after figure content insertion, these per-element spans will be inaccurate.
    # This may impact use cases like citation page number calculation.
    # Additional code may be needed to correct the spans or otherwise infer the page numbers for each citation.
    # The main purpose of the notebook is to show the feasibility of using Content Understanding with Azure Search for RAG chat applications.

    # Validate span_offsets are sorted and strictly increasing
    if span_offsets != sorted(span_offsets) or not all([o < span_offsets[i + 1] for i, o in enumerate(span_offsets) if i < len(span_offsets) - 1]):
        raise ValueError("span_offsets should be sorted and strictly increasing.")

    # Split the content based on the provided spans
    parts = []
    preamble = None
    for i, offset in enumerate(span_offsets):
        if i == 0 and offset > 0:
            preamble = md_content[0:offset]
            parts.append(md_content[offset:span_offsets[i + 1]])
        elif i == len(span_offsets) - 1:
            parts.append(md_content[offset:])
        else:
            parts.append(md_content[offset:span_offsets[i + 1]])

    # Join the parts back together with the figure content inserted
    modified_content = ""
    if preamble:
        modified_content += preamble
    for i, part in enumerate(parts):
        modified_content += f"<!-- FigureContent=\"{figure_contents[i]}\" -->" + part

    return modified_content

def crop_image_from_pdf_page(pdf_path, page_number, bounding_box):
    """
    Crops a region from a given page in a PDF and returns it as an image.

    Args:    
    - pdf_path (pathlib.Path): Path to the PDF file.
    - page_number (int): The page number to crop from (0-indexed).
    - bounding_box (tuple): A tuple of (x0, y0, x1, y1) coordinates for the bounding box.
    
    Returns:
    - PIL.Image: A PIL Image of the cropped area.
    """
    doc = fitz.open(pdf_path)
    page = doc.load_page(page_number)
    
    # Cropping the page. The rect requires the coordinates in the format (x0, y0, x1, y1).
    bbx = [x * 72 for x in bounding_box]
    rect = fitz.Rect(bbx)
    pix = page.get_pixmap(matrix=fitz.Matrix(300 / 72, 300 / 72), clip=rect)
    
    img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
    
    doc.close()

    return img

def format_content_understanding_result(content_understanding_result):
    """
    Formats the JSON output of the Content Understanding result as Markdown for downstream usage in text.
    
    Args:
    - content_understanding_result (dict): A dictionary containing the output from Content Understanding.

    Returns:
    - str: A Markdown string of the result content.
    """
    def _format_result(key, result):
        result_type = result["type"]
        if result_type in ["string", "integer", "number", "boolean"]:
            return f"**{key}**: " + str(result[f'value{result_type.capitalize()}']) + "\n"
        elif result_type == "array":
            return f"**{key}**: " + ', '.join([str(result["valueArray"][i][f"value{r['type'].capitalize()}"]) for i, r in enumerate(result["valueArray"])]) + "\n"
        elif result_type == "object":
            return f"**{key}**\n" + ''.join([_format_result(f"{key}.{k}", result["valueObject"][k]) for k in result["valueObject"]])

    fields = content_understanding_result['result']['contents'][0]['fields']
    markdown_result = ""
    for field in fields:
        markdown_result += _format_result(field, fields[field])

    return markdown_result

In [ ]:
import io
import json
import os
import shutil
from datetime import datetime
from pathlib import Path

# Configuration
CONTENT_TYPE = "textbook"  # Can be changed to other types like "novel", "academic", etc.

# Generate timestamp and setup folder structure
timestamp = datetime.now().strftime('%Y%m%d-%H%M%S')
bookname = file.stem  # Extract filename without extension (e.g., "Venture-deals")
job_folder_name = f"{bookname}_{CONTENT_TYPE}_{timestamp}"

# Create folder structure: content/books/{bookname}_{contenttype}_{timestamp}/
books_root = Path("../content/books")
job_folder = books_root / job_folder_name
input_folder = job_folder / "input"
processed_folder = job_folder / "processed"

# Create directories
input_folder.mkdir(parents=True, exist_ok=True)
processed_folder.mkdir(parents=True, exist_ok=True)

# Copy original PDF to input folder
original_pdf_dest = input_folder / file.name
shutil.copy2(file, original_pdf_dest)
print(f"✓ Created job folder: {job_folder}")
print(f"✓ Copied input PDF to: {original_pdf_dest}")

# Initialize empty structures for chapter data (will be populated by LLM later)
chapters = []
document_metadata = {
    "title": "",
    "summary": ""
}

print("\n" + "=" * 70)
print("STEP 1: Extract layout and figures with Document Intelligence")
print("=" * 70)
print(f"Started: {datetime.now().strftime('%H:%M:%S')}")

# Run Content Understanding on each figure, format figure contents, and insert figure contents into corresponding document locations
with open(file, 'rb') as f:
    pdf_bytes = f.read()
    print(f"  → Document size: {len(pdf_bytes) / 1024 / 1024:.2f} MB")

    document_intelligence_client = DocumentIntelligenceClient(
        endpoint=AZURE_AI_SERVICE_ENDPOINT,
        api_version=AZURE_DOCUMENT_INTELLIGENCE_API_VERSION,
        credential=credential,
        output=str('figures')
    )

    print("  → Analyzing document layout...")
    poller = document_intelligence_client.begin_analyze_document(
        "prebuilt-layout",
        AnalyzeDocumentRequest(bytes_source=pdf_bytes),
        output=[str('figures')],
        features=['ocrHighResolution'],
        output_content_format="markdown"
    )

    result: AnalyzeResult = poller.result()
    
    md_content = result.content

    print(f"✓ Extracted document content ({len(md_content)} characters)")
    print(f"✓ Detected {len(result.figures) if result.figures else 0} figures")
    print(f"  Completed at: {datetime.now().strftime('%H:%M:%S')}")

    print("\n" + "=" * 70)
    print("STEP 3: Analyze figures with Content Understanding")
    print("=" * 70)
    print(f"Started: {datetime.now().strftime('%H:%M:%S')}")

    figure_contents = []
    if result.figures:
        total_figures = len(result.figures)
        print(f"Extracting figure contents with Content Understanding image analyzer...")
        print(f"Total figures to process: {total_figures}")
        
        for figure_idx, figure in enumerate(result.figures):
            for region in figure.bounding_regions:
                    # Uncomment the below to print out the bounding regions of each figure
                    # print(f"Figure {figure_idx + 1} body bounding regions: {region}")
                    # To learn more about bounding regions, see https://aka.ms/bounding-region
                    bounding_box = (
                            region.polygon[0],  # x0 (left)
                            region.polygon[1],  # y0 (top
                            region.polygon[4],  # x1 (right)
                            region.polygon[5]   # y1 (bottom)
                        )
            page_number = figure.bounding_regions[0]['pageNumber']
            cropped_img = crop_image_from_pdf_page(file, page_number - 1, bounding_box)

            os.makedirs("figures", exist_ok=True)

            figure_filename = f"figure_{figure_idx + 1}.png"
            # Full path for the file
            figure_filepath = os.path.join("figures", figure_filename)

            # Save the figure
            cropped_img.save(figure_filepath)
            bytes_io = io.BytesIO()
            cropped_img.save(bytes_io, format='PNG')
            cropped_img = bytes_io.getvalue()

            # Collect formatted content from the figure
            content_understanding_response = content_understanding_client.begin_analyze(ANALYZER_ID, figure_filepath)
            content_understanding_result = content_understanding_client.poll_result(content_understanding_response, timeout_seconds=1000)
            figure_content = format_content_understanding_result(content_understanding_result)
            figure_contents.append(figure_content)
            
            # Progress indicator - show every 10 figures or every figure for small sets
            if total_figures <= 20 or (figure_idx + 1) % 10 == 0 or (figure_idx + 1) == total_figures:
                progress_pct = ((figure_idx + 1) / total_figures) * 100
                print(f"  ✓ Figure {figure_idx + 1}/{total_figures} analyzed (page {page_number}) - {progress_pct:.1f}% complete")

        print(f"\n✓ Analyzed {len(figure_contents)} figures")
        print(f"  Completed at: {datetime.now().strftime('%H:%M:%S')}")

        # Insert figure content into corresponding location in document
        print("  → Inserting figure descriptions into document...")
        md_content = insert_figure_contents(md_content, figure_contents, [f.spans[0]["offset"] for f in result.figures])
    
    print("\n" + "=" * 70)
    print("STEP 4: Save enhanced document with chapter metadata")
    print("=" * 70)

    # Save enhanced document with both figure descriptions and chapter metadata
    result.content = md_content
    output = {
        "analyzeResult": result.as_dict(),
        "chapters": chapters,
        "document_metadata": document_metadata,
        "processing_info": {
            "timestamp": timestamp,
            "job_folder": str(job_folder),
            "bookname": bookname,
            "content_type": CONTENT_TYPE,
            "original_file": str(file),
            "total_chapters": len(chapters),
            "total_figures": len(figure_contents)
        }
    }
    output_json = json.dumps(output, indent=2)
    
    # Save cache JSON with proper naming: {bookname}_{contenttype}_cache_{timestamp}.json
    cache_filename = f"{bookname}_{CONTENT_TYPE}_cache_{timestamp}.json"
    cache_path = processed_folder / cache_filename
    with open(cache_path, 'w', encoding='utf-8') as f:
        f.write(output_json)
    
    # Save markdown with proper naming: {bookname}_{contenttype}_markdown_{timestamp}.md
    markdown_filename = f"{bookname}_{CONTENT_TYPE}_markdown_{timestamp}.md"
    markdown_path = processed_folder / markdown_filename
    with open(markdown_path, 'w', encoding='utf-8') as f:
        f.write(f"# {document_metadata.get('title', bookname)}\n\n")
        f.write(f"**Document**: {file.name}\n")
        f.write(f"**Processed**: {timestamp}\n")
        f.write(f"**Content Type**: {CONTENT_TYPE}\n\n")
        if document_metadata.get('summary'):
            f.write(f"**Summary**: {document_metadata.get('summary')}\n\n")
        f.write(f"**Total Chapters**: {len(chapters)}\n")
        f.write(f"**Total Figures**: {len(figure_contents)}\n\n")
        f.write("---\n\n")
        f.write(md_content)
    
    # Save metadata summary as separate JSON for quick reference
    metadata_filename = f"{bookname}_{CONTENT_TYPE}_metadata_{timestamp}.json"
    metadata_path = processed_folder / metadata_filename
    metadata_summary = {
        "document_title": document_metadata.get('title', ''),
        "document_summary": document_metadata.get('summary', ''),
        "bookname": bookname,
        "content_type": CONTENT_TYPE,
        "timestamp": timestamp,
        "original_file": file.name,
        "total_chapters": len(chapters),
        "total_figures": len(figure_contents),
        "total_characters": len(md_content),
        "chapters": chapters,
        "job_folder": str(job_folder),
        "files": {
            "input_pdf": str(original_pdf_dest),
            "cache": str(cache_path),
            "markdown": str(markdown_path),
            "metadata": str(metadata_path)
        }
    }
    with open(metadata_path, 'w', encoding='utf-8') as f:
        json.dump(metadata_summary, f, indent=2)

    print(f"\n✓ All files saved to: {job_folder}")
    print(f"\n  Input:")
    print(f"    - {original_pdf_dest.name}")
    print(f"\n  Processed:")
    print(f"    - {cache_filename} ({len(output_json) / 1024 / 1024:.2f} MB)")
    print(f"    - {markdown_filename} ({len(md_content) / 1024:.1f} KB)")
    print(f"    - {metadata_filename}")
    print(f"\n  Summary:")
    print(f"    - {len(chapters)} chapters with metadata")
    print(f"    - {len(figure_contents)} figures analyzed")
    print(f"    - {len(md_content):,} characters in markdown")
    print(f"\nTotal process completed at: {datetime.now().strftime('%H:%M:%S')}")

✓ Created job folder: ../content/books/Venture-deals_textbook_20251004-120709
✓ Copied input PDF to: ../content/books/Venture-deals_textbook_20251004-120709/input/Venture-deals.pdf

STEP 1: Extract chapters with Content Understanding
Started: 12:07:09
  → Sending document for chapter analysis...
  → Waiting for chapter extraction to complete (this may take a few minutes)...
  → Waiting for chapter extraction to complete (this may take a few minutes)...
  → Chapter extraction completed at 12:10:55

✓ Extracted 1 chapters
✓ Document: Venture Deals: Be Smarter than Your Lawyer and Venture Capitalist, Third Edition...
  1. 12.: How to Raise Money (Pages 0-0, Level 1)

STEP 2: Extract layout and figures with Document Intelligence
Started: 12:10:55
  → Document size: 1.58 MB
  → Analyzing document layout...
  → Chapter extraction completed at 12:10:55

✓ Extracted 1 chapters
✓ Document: Venture Deals: Be Smarter than Your Lawyer and Venture Capitalist, Third Edition...
  1. 12.: How to Raise

In [12]:
# To load a previously cached result, uncomment and specify the cache file path:
# Example: cache_path = Path("../content/books/Venture-deals_textbook_20251004-143022/processed/Venture-deals_textbook_cache_20251004-143022.json")
# with open(cache_path, 'r', encoding='utf-8') as f:
#     output_json = f.read()

# Otherwise, use the data from the previous cell
document_data = json.loads(output_json)
document_content = document_data['analyzeResult']['content']
chapters = document_data.get('chapters', [])
document_metadata = document_data.get('document_metadata', {})
processing_info = document_data.get('processing_info', {})

print(f"✓ Loaded document with {len(chapters)} chapters")
print(f"  Document: {document_metadata.get('title', 'N/A')[:80]}...")
print(f"  Content size: {len(document_content):,} characters")
if processing_info:
    print(f"  Job folder: {processing_info.get('job_folder', 'N/A')}")
    print(f"  Processed: {processing_info.get('timestamp', 'N/A')}")

# Display first 20 chapters with summaries
print(f"\n{'='*70}")
print(f"CHAPTER SUMMARIES (showing first 20 of {len(chapters)})")
print(f"{'='*70}\n")
for i, chapter in enumerate(chapters[:20], 1):
    chapter_num = f"{chapter['number']}: " if chapter['number'] else ""
    print(f"{i}. {chapter_num}{chapter['title']}")
    print(f"   Pages: {chapter['page_start']}-{chapter['page_end']} | Level: {chapter['level']}")
    if chapter.get('summary'):
        print(f"   Summary: {chapter['summary'][:200]}..." if len(chapter['summary']) > 200 else f"   Summary: {chapter['summary']}")
    if chapter.get('key_topics'):
        print(f"   Topics: {', '.join(chapter['key_topics'][:3])}")
    print()

# Display preview of markdown content
print(f"\n{'='*70}")
print(f"MARKDOWN CONTENT PREVIEW (first 2000 characters)")
print(f"{'='*70}\n")
print(document_content[:2000])
if len(document_content) > 2000:
    print(f"\n... ({len(document_content) - 2000:,} more characters)")
    print(f"\nFull content available in: {processing_info.get('job_folder', 'processed folder')}")

Loaded document with 4 chapters


## Chunk text by splitting with Markdown header splitting and recursive character splitting
This is a simple starting point. Feel free to give your own chunking strategies a try!

In [13]:
from langchain_text_splitters import MarkdownHeaderTextSplitter
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.schema import Document

# Configure langchain text splitting settings
EMBEDDING_CHUNK_SIZE = 512
EMBEDDING_CHUNK_OVERLAP = 20
headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
    ("###", "Header 3")
]

# First split text using Markdown headers
text_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on, strip_headers=False)
chunks = text_splitter.split_text(document_content)

# Then further split the text using recursive character text splitting
char_text_splitter = RecursiveCharacterTextSplitter(separators=["<!--", "\n\n", "#"], chunk_size=EMBEDDING_CHUNK_SIZE, chunk_overlap=EMBEDDING_CHUNK_OVERLAP, is_separator_regex=True)
chunks = char_text_splitter.split_documents(chunks)

# Helper function to find which chapter a chunk belongs to
def find_chapter_for_chunk(chunk_content, chapters):
    """
    Determines which chapter a chunk belongs to based on content analysis.
    Returns chapter metadata or None.
    """
    # Check if the chunk contains chapter titles or headers
    for chapter in chapters:
        if chapter['title'].lower() in chunk_content.lower():
            return chapter
    return None

# Enrich chunks with chapter metadata
enriched_chunks = []
for chunk in chunks:
    # Find which chapter this chunk belongs to
    chapter = find_chapter_for_chunk(chunk.page_content, chapters)
    
    # Add chapter metadata to chunk
    if chapter:
        chunk.metadata.update({
            "chapter_title": chapter['title'],
            "chapter_number": chapter['number'],
            "chapter_level": chapter['level'],
            "chapter_summary": chapter['summary'],
            "chapter_topics": ", ".join(chapter['key_topics']),
            "page_start": chapter['page_start'],
            "page_end": chapter['page_end']
        })
    
    # Add document-level metadata
    chunk.metadata.update({
        "document_title": document_metadata.get('title', ''),
        "document_summary": document_metadata.get('summary', '')
    })
    
    enriched_chunks.append(chunk)

print(f"Number of chunks: {len(enriched_chunks)}")
print(f"Chunks with chapter metadata: {sum(1 for c in enriched_chunks if 'chapter_title' in c.metadata)}")

# Show sample chunk with metadata
if enriched_chunks:
    print(f"\nSample chunk metadata:")
    if 'chapter_title' in enriched_chunks[0].metadata:
        print(f"  Chapter: {enriched_chunks[0].metadata.get('chapter_title')}")
        print(f"  Topics: {enriched_chunks[0].metadata.get('chapter_topics')}")
    print(f"  Document: {enriched_chunks[0].metadata.get('document_title', 'N/A')}")

Number of chunks: 302
Chunks with chapter metadata: 302

Sample chunk metadata:
  Chapter: 
  Topics: 
  Document: 2024 ARTICLE IV CONSULTATION-PRESS RELEASE; STAFF REPORT; AND STATEMENT BY THE EXECUTIVE DIRECTOR FOR THE UNITED STATES


## Query vector index to retrieve relevant documents

In [14]:
from langchain_openai import AzureOpenAIEmbeddings
from langchain.vectorstores.azuresearch import AzureSearch

aoai_embeddings = AzureOpenAIEmbeddings(model=AZURE_OPENAI_EMBEDDING_DEPLOYMENT_NAME,
                                        azure_endpoint=AZURE_OPENAI_ENDPOINT,
                                        azure_ad_token_provider=token_provider,
                                        api_version=AZURE_OPENAI_EMBEDDING_API_VERSION)

vector_store = AzureSearch(
    azure_search_endpoint=AZURE_SEARCH_ENDPOINT,
    azure_search_key=None,
    index_name=AZURE_SEARCH_INDEX_NAME,
    embedding_function=aoai_embeddings.embed_query
)

# This is a one-time operation to add the documents to the vector store. Comment out this line if you are re-running this cell with the same index.
print(f"Indexing {len(enriched_chunks)} enriched chunks with chapter metadata...")
vector_store.add_documents(documents=enriched_chunks)
print("✓ Indexing complete")

# Set up the retriever that will be used to query the index for similar documents
retriever = vector_store.as_retriever(search_type="similarity")
print("✓ Vector store ready for chapter-aware search!")

Indexing 302 enriched chunks with chapter metadata...
✓ Indexing complete
✓ Vector store ready for chapter-aware search!
✓ Indexing complete
✓ Vector store ready for chapter-aware search!


In [15]:
# Retrieve relevant documents
query = "What was the crude oil production in 2019?"
retrieved_docs = retriever.invoke(query)

print(f"Query: {query}")
print(f"Retrieved {len(retrieved_docs)} documents\n")

# Print retrieved documents with chapter information
for i, doc in enumerate(retrieved_docs, 1):
    print(f"Document {i}:")
    print(f"  ID: {doc.metadata.get('id', 'N/A')}")
    if 'chapter_title' in doc.metadata:
        print(f"  Chapter: {doc.metadata.get('chapter_title')}")
        print(f"  Chapter Topics: {doc.metadata.get('chapter_topics')}")
        print(f"  Pages: {doc.metadata.get('page_start')}-{doc.metadata.get('page_end')}")
    print(f"  Content: {doc.page_content[:200]}...")
    print("=" * 70)

Query: What was the crude oil production in 2019?
Retrieved 4 documents

Document 1:
  ID: ZDVhZGI4NTktN2Q1NS00NGQ3LThkZTEtN2VkYzA5ZTM1MGJh
  Content: <!-- FigureContent="**Title**: U.S. Energy Production
**ChartType**: line
**TopicKeywords**: Business and finance, Energy, Environment
**DetailedDescription**: The chart displays the production levels...
Document 2:
  ID: ODdiODdmZmYtMjBjNC00NWQ0LTkxZDUtN2RiYjYyODQ2YmJm
  Content: <!-- FigureContent="**Title**: U.S. Energy Production
**ChartType**: line
**TopicKeywords**: Business and finance, Energy, Environment
**DetailedDescription**: The chart shows the production levels of...
Document 3:
  ID: NGYxN2FiYWUtYjY0My00ZWYyLTg2MzItM2FlOTQxZDRhMDRj
  Chapter: 
  Chapter Topics: 
  Pages: 0-0
  Content: <!-- FigureContent="**Title**: U.S. Energy Production
**ChartType**: line
**TopicKeywords**: Business and finance, Energy, Environment
**DetailedDescription**: The chart displays the production levels...
Document 4:
  ID: NjRlNGNiMDgtMGU4Yy0

## Generate answer to query

In [16]:
# Define system prompt template for chat model
prompt = """
You are an expert in document analysis. You are proficient in reading and analyzing technical reports. You are good at numerical reasoning and have a good understanding of financial concepts. You are given a question which you need to answer based on the references provided. To answer this question, you may first read the question carefully to know what information is required or helpful to answer the question. Then, you may read the references to find the relevant information.

If you find enough information to answer the question, you can first write down your thinking process and then provide a concise answer at the end.
If you find that there is not enough information to answer the question, you can state that there is insufficient information.
If you are not able or sure how to answer the question, say that you are not able to answer the question.
Do not provide any information that is not present in the references.
References are in markdown format, you may follow the markdown syntax to better understand the references.

---
References:
{context}
---

Now, here is the question:
---
Question:
{question}
---
Thinking Process::: 
Answer::: 
"""

# Helper function to generate the formatted context from each retrieved document
def generate_context(chunks):
    context = []
    for i, chunk in enumerate(chunks):
        s = (f'Source {i} Metadata: {chunk.metadata}\n'
                f'Source {i} Content: {chunk.page_content}')
        context.append(s)
    context = '\n---\n'.join(context)
    return context

# Remove redundant chunks
appeared = set()
unique_chunks = []
for chunk in retrieved_docs:
    chunk_id = chunk.metadata['id']
    if chunk_id not in appeared:
        appeared.add(chunk_id)
        unique_chunks.append(chunk)
context = generate_context(unique_chunks)

# Format the prompt with the provided query and formatted context
prompt = prompt.format(question=query,
                       context=context)

In [17]:
from langchain_openai import AzureChatOpenAI

chat_llm = AzureChatOpenAI(model=AZURE_OPENAI_CHAT_DEPLOYMENT_NAME,
                            azure_endpoint=AZURE_OPENAI_ENDPOINT,
                            azure_ad_token_provider=token_provider,
                            api_version=AZURE_OPENAI_CHAT_API_VERSION,
                            temperature=0.7)

# Print the LLM's answer to the query with the retrieved documents as additional context
answer = chat_llm.invoke(prompt)
print(answer.content)

Thinking process:
- The provided chart's data table lists crude oil production by quarter.  
- For 2019Q1 the table shows crude oil production of 12 million barrels per day.

Answer:
Crude oil production in 2019 was 12 million barrels per day.
